In [ ]:
import os
import sys
import requests
from mimetypes import guess_type
import urllib.parse
import time
import pandas as pd
import logging

In [ ]:
class pypeertube(object):
    """Batch uploads videos to a peertube instance"""
    def __init__(self):
        from auth_data import auth_data,config,file_extensions
        self.config = config
        self.file_extensions = file_extensions
        self.auth_data = auth_data
        self.auth_data["grant_type"] = "password"
        self.auth_data["response_type"] = "code"
        
        super(pypeertube, self).__init__()
        
        logging.basicConfig(
                            #filename="event.log",
                            level=logging.INFO,
                            format='%(asctime)s %(levelname)s %(processName)s %(message)s', 
                            datefmt='%d.%m.%Y %H:%M:%S %p'
                            ) 
        
        if not self.config["channelID"]:
            self.config["channelID"] = requests.get('{0}{1}/{2}'.format(self.config["site"], 
                                                                        '/api/v1/video-channels', 
                                                                        config["channel"]
                                                                       )).json()['id']
            logging.info("channelID: {}".format(self.config["channelID"]))
    
        self.upload_url = self.config["site"] + "/api/v1/videos/upload"
        logging.info("upload_url: {}".format(self.upload_url))

        self.dataframe_file = "dataframe.pkl"
        logging.info("dataframe_file: {}".format(self.dataframe_file))

        self.file_mimetype = "video/mp4"
        logging.info("file_mimetype: {}".format(self.file_mimetype))
        
        self.oauth_url = config["site"] + "/api/v1/oauth-clients/local"
        logging.info("oauth_url: {}".format(self.oauth_url))

        self.auth_data["client_id"] = requests.get(self.oauth_url).json()["client_id"]
        logging.info("client_id: {}".format(self.auth_data["client_id"]))

        self.auth_data["client_secret"] = requests.get(self.oauth_url).json()["client_secret"]
        logging.info("client_secret: {}".format(self.auth_data["client_secret"]))

        self.token_url = self.config["site"] + "/api/v1/users/token"
        logging.info("token_url: {}".format(self.token_url))

        self.token_data = requests.post(self.token_url, data=self.auth_data).json() #access_token,token_type,expires_in,refresh_token
        logging.info(str(self.token_data))
        logging.info("access_token: {}".format(self.token_data["access_token"]))
        logging.info("token_type: {}".format(self.token_data["token_type"]))
        logging.info("expires_in: {}".format(self.token_data["expires_in"]))
        logging.info("refresh_token: {}".format(self.token_data["refresh_token"]))
        
        self.upload_headers = {'Authorization': 'Bearer {0}'.format(self.token_data["access_token"])}
        
        self.upload_data = {
                'channelId': self.config["channelID"],
                'privacy': self.config["privacy"],
                'commentsEnabled': self.config["commentsEnabled"],
                #'name': filename
               }
        
    def import_dataframe(self):
        
        #Pandas loading dataframe from file
        try:
            self.df = pd.read_pickle(self.dataframe_file)
        except Exception as e:
            logging.error(e)
            logging.info("Recreating dataframe")
            logging.info("Finding every videofile in \"{}\"".format(self.config["path"]))
            self.df = self.create_list_of_video_files()
            if self.df:
                logging.info("Found {} videos in \"{}\"".format(len(self.df),self.config["path"]))
                logging.info("Saved data in {}".format(self.dataframe_file))
        
        
    def create_list_of_video_files(self):
        files_list = list()
        for root, subdirs, files in os.walk(self.config["path"]):            
            for file in files:
                skippme = False
                if file not in files_list:
                    file_location = root+"/"+file
                    file_mimetype = guess_type(file_location)[0]
                    filename = file

                    #'file_mimetype': 'video/x-flv'}
                    #'file_mimetype': 'video/mp4'}
                    #'file_mimetype': 'video/x-ms-wmv'}
                    #'file_mimetype': 'video/webm'}
                    #'file_mimetype': 'video/x-msvideo'}
                    #'file_mimetype': 'video/x-matroska'}
                    #'file_mimetype': 'video/3gpp'}
                    #'file_mimetype': 'video/quicktime'}
                    #'file_mimetype': 'video/x-ms-asf'}
                    #'file_mimetype': 'video/mpeg'}
                    
                    if file_mimetype:
                        if "video" in file_mimetype:
                            
                            for exclude in self.config["exclude_list"]:
                                exclude = exclude.replace(" ","")
                                if exclude != "":
                                    if exclude in file_location:
                                        logging.info("Excluding \"{}\" from uploading".format(file_location))
                                        skippme = True
                            if not skippme:
                                files_list.append({"filename":filename,
                                                   "file_location":file_location,
                                                   "file_mimetype":file_mimetype,
                                                   "status":"queued"
                                                  })
                    #status = queued|in progress|failed|completed

        #Create pandas dataframe
        self.df = pd.DataFrame(files_list)

        #Save pandas dataframe
        self.df.to_pickle(self.dataframe_file)

    
    def search(self,pattern):
        pattern_formatted = urllib.parse.quote(pattern)
        pattern_cleaned_file_extension = urllib.parse.quote(self.delete_file_extensions_from_videoname(pattern))
        
        time.sleep(0.1)
        try:
            res = requests.get(
                self.config["site"] + "/api/v1/search/videos?start=0&count=10&search={}&sort=-match".format(pattern_formatted)
                        ).json()
            if res:
                for result in res["data"]:
                    if result["name"] == pattern:
                        return result   
            else:
                res = requests.get(
                self.config["site"] + "/api/v1/search/videos?start=0&count=10&search={}&sort=-match".format(pattern_cleaned_file_extension)
                        ).json()
                if res:
                    for result in res["data"]:
                        if result["name"] == pattern:
                            return result 
                        
        except Exception as e:
            logging.error(e)
            logging.error("unable to search for {} an {}".format(pattern,self.config["site"]))
            
        
    def sync_progress(self):
        
        if not self.df:
            self.import_dataframe()
            
        for index, row in self.df.iterrows():
            logging.info("Video number {} from {}".format(index,len(self.df)))
            video_title = row["filename"]
            try:
                res = self.search(video_title)

                for item in res["data"]:
                    logging.info("Returned search results. Comparing search result \"{}\" with \"{}\"".format(
                                                                                        item["name"],video_title
                                                                                    ))
                    if item["name"] == video_title:
                        logging.info("found this video: {}. Marked it as completed".format(video_title))
                        row["status"] = "completed"
            except:
                logging.error("unable to search for {}".format(video_title))
                logging.error(res)

            time.sleep(0.2)

        #Save pandas dataframe
        self.df.to_pickle(self.dataframe_file)
        
    def delete_file_extensions_from_videoname(self,name):    
        for extension in self.file_extensions :
            if extension in name:
                return name.replace(extension, '')

    def get_video_metadata(self,video_id):
        video_id = "cb0391fa-051f-4c26-97f1-87b547ca84a6"
        url = "{}/api/v1/videos/{}".format(self.config["site"],video_id)
        res = requests.get(url)
        return res.json()

    def set_video_metadata(self,video_id,payload):
        url = "{}/api/v1/videos/{}".format(self.config["site"],video_id)
        return requests.put(url,
                            headers = self.upload_headers,
                            data = payload
                           )

    def scan_website_for_metadata(self):
        pass
    
        #find every video
        #find every tag in every video
        #extend dataframe with new infos


    def learn_tags_from_folder_structure_and_update_videos(self):
        
        for index,row in self.df.iterrows():
            filename = self.df.iloc[index]["filename"]
            file_location = self.df.iloc[index]["file_location"]

            #remove initial path from file location, remove filename from filelocation 
            #and split the rest on every "/" into the tags-list
            tags = file_location.replace(self.config["path"],"").replace(filename,"").split("/")

            for i in range(len(tags)):
                tag = self.string_cleanup(tags[i]) #removes special characters and spaces from tags
                tags[i] = tag[:20]

            tags = list(filter(None,tags))
            uuid = None
            res = self.search(filename)
            if res:
                uuid = res["uuid"]
            try:
                if tags and uuid:
                    self.set_video_metadata(uuid,{"tags":tags})
            except Exception as e:
                logging.error("Error: {}".format(e))
                logging.info("Filename: {}".format(filename))
                logging.info("File Location: {}".format(file_location))
                logging.info("Tags: {}".format(tags))
                logging.info("UUID: {}".format(uuid))
            finally:
                logging.info("{} - {}".format(tags,uuid))

            
        
    def get_videos(self,username,count,start):
        url = "{}/api/v1/accounts/{}/videos".format(self.config["site"],username)
        return requests.get(
                            url,
                            params={'count': count, 'start': start}
                            )

    def string_cleanup(self,string):
        #removes special characters (including spaces) from string
        new_string = ""
        for char in string:
            if char.isalnum():
                new_string += char
        return new_string

    
    def upload_videos(self):
        if not self.df:
            self.read_videos_from_directory()
            
        for index, row in self.df.iterrows():
            logging.info("Video {} of {} is being uploaded.".format(index+1,len(self.df)))
            self.upload_data["name"] = self.delete_file_extensions_from_videoname(row["filename"])
            if row["status"] != "completed":
        
                logging.info("Uploading video \"{}\" from \"{}\" with mimetype \"{}\"".format(row["filename"],
                                                                                              row["file_location"],
                                                                                              row["file_mimetype"]
                                                                                             ))

                try:
                    self.upload_result = None
                    with open(row["file_location"], 'rb') as f:
                        self.upload_result = requests.post(self.upload_url,
                                                      headers=self.upload_headers,
                                                      data=self.upload_data,
                                                      files={"videofile": (row["filename"], 
                                                                           f, 
                                                                           row["file_mimetype"])})
                    if app.upload_result.json()["video"]:
                        logging.info("Finished upload successfully")
                        #self.df.iloc[index]["id"] = app.upload_result.json()["video"]["id"]
                        #self.df.iloc[index]["uuid"] = app.upload_result.json()["video"]["uuid"]
                        self.df.iloc[index]["status"] = "completed"
                    
                        #Save pandas dataframe
                        self.df.to_pickle(self.dataframe_file)
                    else:
                        if self.upload_result:
                            logging.error("Aborting upload. Error: {}".format(self.upload_result.text))
                        else:
                            logging.error("Aborting upload")
                    
                except Exception as e:
                    logging.error(e)
                    if self.upload_result:
                            logging.error("Aborting upload. Error: {}".format(self.upload_result.text))
                    else:
                            logging.error("Aborting upload")


            else:
                logging.info("Skipping video \"{}\". It's already marked as completed.".format(row["filename"]))
    

In [ ]:
app = pypeertube()

In [ ]:
app.import_dataframe()

In [ ]:
app.df

In [ ]:
# count = 10
# start = 0
# videos = app.get_videos("user",count,start).json()

In [ ]:
# print(len(videos["data"]))
# for video in videos["data"]:
#     print(video["name"])
    